In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("scr/Datasets/movies_dataset_ETL.csv")

In [5]:
data.columns

Index(['id', 'title', 'genres', 'belongs_to_collection', 'original_language',
       'spoken_languages', 'production_companies', 'production_countries',
       'release_date', 'release_year', 'revenue', 'budget', 'return',
       'popularity', 'vote_average', 'vote_count', 'runtime', 'status',
       'overview', 'tagline', 'director', 'cast'],
      dtype='object')

In [6]:
df = data[['id', 'title', 'release_date', 'release_year','popularity', 'vote_average', 'vote_count', 'status', 'cast', 'director', 'return', 'budget', 'revenue']].copy()

In [7]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
df['release_month'] = pd.DatetimeIndex(df['release_date']).month
df['release_weekday'] = pd.DatetimeIndex(df['release_date']).weekday

API 1

In [9]:
def cantidad_filmaciones_mes(mes:str):
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente'''

    mes = str.lower(mes)
    m_dict = {
        'enero': 1,
        'febrero': 2,
        'marzo': 3,
        'abril': 4,
        'mayo': 5,
        'junio': 6,
        'julio': 7,
        'agosto': 8,
        'septiembre': 9,
        'octubre': 10,
        'noviembre': 11,
        'diciembre': 12
        }

    df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

    count = df[df['release_month'] == m_dict[mes]].shape[0]
    
    return {"mes":str(mes), "cantidad_filmaciones":int(count)}

In [10]:
cantidad_filmaciones_mes("julio")

{'mes': 'julio', 'cantidad_filmaciones': 2646}

API 2

In [19]:
def cantidad_filmaciones_dia(dia:str):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrenaron ese dia historicamente'''

    dia = str.lower(dia)
    d_dict = {
        'lunes': 0,
        'martes': 1,
        'miercoles': 2,
        'jueves': 3,
        'viernes': 4,
        'sabado': 5,
        'domingo': 6,
        }
    
    #df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

    dff = df[df['status'] == 'Released']

    count = dff.loc[dff['release_weekday'] == d_dict[dia]].shape[0]
    
    return {"dia":str(dia), "cantidad_filmaciones": str(count)}

In [12]:
df['status'].unique()

array(['Released', nan, 'Rumored', 'Post Production', 'In Production',
       'Planned', 'Canceled'], dtype=object)

In [20]:
cantidad_filmaciones_dia("domingo")

{'dia': 'domingo', 'cantidad_filmaciones': '3584'}

API 3

In [12]:
def score_titulo(titulo:str):
    '''Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score'''

    titulo = str.lower(titulo)
    
    dff = df[df['title'].apply(str.lower) == titulo].sort_values('release_year',ascending=False)
    
    dff.rename(columns={'title': 'titulo', 'release_year': 'anio_estreno','popularity': 'popularidad'}, inplace=True)
    dff = dff[['titulo', 'anio_estreno', 'popularidad']]
    
    return dff.iloc[0].to_dict()

In [13]:
score_titulo("blackout")

{'titulo': 'Blackout', 'anio_estreno': 2015, 'popularidad': 0.423677}

API 4

In [14]:
def votos_titulo(titulo:str):
    '''Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. 
    La misma variable deberá de contar con al menos 2000 valoraciones, 
    caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.'''

    titulo = str.lower(titulo)

    dff = df[df['title'].apply(str.lower) == titulo].sort_values('release_year',ascending=False)
    
    dff.rename(columns={'title': 'titulo', 'release_year': 'anio','vote_count': 'voto_total', 'vote_average': 'voto_promedio'}, inplace=True)
    dff = dff[['titulo', 'anio', 'voto_total', 'voto_promedio']]


    count = dff['voto_total'].sum()
    avg = dff['voto_promedio'].mean()
    year = dff['anio'].iloc[0]
    
    if count >= 2000:
        return {"titulo":str(titulo), 'anio':int(year), "voto_total":int(count),"voto_promedio":int(avg)}
    else:
        return {"titulo":str(titulo), 'mensaje': 'No es posible un resultado ya que el título seleccionado tiene pocas valoraciones.'}

In [15]:
votos_titulo("Blackout")

{'titulo': 'blackout',
 'mensaje': 'No es posible un resultado ya que el título seleccionado tiene pocas valoraciones.'}

In [16]:
votos_titulo("Toy Story")

{'titulo': 'toy story', 'anio': 1995, 'voto_total': 5415, 'voto_promedio': 7.7}

API 5

In [17]:
def get_actor(nombre_actor:str):
    '''Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, la cantidad de películas que en las que ha participado y el promedio de retorno'''

    nombre_actor = str.lower(nombre_actor)

    mask = df['cast'].apply(str.lower).str.find(nombre_actor) != -1
    dff = df[mask]

    count = dff.shape[0]
    retorno_tot = dff['return'].sum()
    retorno_prom = dff['return'].mean()

    return {"actor": str(nombre_actor), "cantidad_filmaciones": int(count),"retorno_total": int(retorno_tot), 'retorno_promedio': int(retorno_prom)}

In [18]:
get_actor('Tom Hanks')

{'actor': 'tom hanks',
 'cantidad_filmaciones': 71,
 'retorno_total': 178.8494960552024,
 'retorno_promedio': 2.5190069866929914}

API 6

In [19]:
def get_director(nombre_director:str):
    ''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.'''

    nombre_director = str.lower(nombre_director)

    dff = df.dropna(subset='director')
    mask = dff['director'].apply(str.lower).str.find(nombre_director) != -1
    dff = dff[mask]  
    
    retorno = dff['return'].sum()

    peliculas = []

    for i in range(dff.shape[0]):
        dic = {}
        dic['title'] = str(dff['title'].iloc[i])
        dic['release_year'] = str(dff['release_year'].iloc[i])
        dic['return'] = str(dff['return'].iloc[i])
        dic['budget'] = str(dff['budget'].iloc[i])
        dic['revenue'] = str(dff['revenue'].iloc[i])
        peliculas.append(dic)

    

    #return {"director":str(nombre_director), "retorno_promedio_por_filmacion":str(retorno),"peliculas":peliculas}

    return {'director':str(nombre_director), 'retorno_total_director':int(retorno), 
    'peliculas':dff['title'].tolist(), 'anio':dff['release_year'].tolist(), 'retorno_pelicula':dff['return'].tolist(), 
    'budget_pelicula':dff['budget'].tolist(), 'revenue_pelicula':dff['revenue'].tolist()}


In [20]:
get_director('Allison Anders')

{'director': 'allison anders',
 'retorno_total_director': 1.075,
 'peliculas': ['Four Rooms',
  'Mi Vida Loca',
  'Grace of My Heart',
  'Sugar Town',
  'Things Behind the Sun',
  'Gas Food Lodging',
  'Border Radio',
  'Ring of Fire',
  'A Crush on You'],
 'anio': [1995, 1994, 1996, 1999, 2001, 1992, 1987, 2013, 2011],
 'retorno_pelicula': [1.075, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'budget_pelicula': [4000000, 0, 0, 0, 0, 0, 0, 0, 0],
 'revenue_pelicula': [4300000, 0, 0, 0, 0, 1345236, 0, 0, 0]}

API 7

Modelo de ML

In [21]:
df.to_csv('scr/Datasets/movies_dataset_ETL_toAPI.csv', index=False)